## Background

Day Spa is a spa and wellness center that offer their clients to relaxation stations as well as access to fitness clubs  including use of the steam rooms, cardio and strength-training equipment.

The company keeps track of clients entry data and they would like to make use of this data to determine how many staff they should have in the spa center. 
We have one table DaySpaVisit that keeps track of when a client enter and leave as well as the number of duration they spend in the center

In [60]:
#relevant installs
import pandas as pd 
import pyodbc
import sqlalchemy as db

#Connect to SQL Server
cnxn_str = ("Driver={SQL Server};"
            "Server=DESKTOP-5FBFTI9\SQLEXPRESS;"
            "Database=MyDb;")
cnxn = pyodbc.connect(cnxn_str)

#Show Table Schema
query = "SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'DaySpaVisit';" 
x =pd.read_sql(query, cnxn)
x

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,ORDINAL_POSITION,COLUMN_DEFAULT,IS_NULLABLE,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,CHARACTER_OCTET_LENGTH,...,DATETIME_PRECISION,CHARACTER_SET_CATALOG,CHARACTER_SET_SCHEMA,CHARACTER_SET_NAME,COLLATION_CATALOG,COLLATION_SCHEMA,COLLATION_NAME,DOMAIN_CATALOG,DOMAIN_SCHEMA,DOMAIN_NAME
0,MyDb,dbo,DaySpaVisit,CustomerID,1,None,NO,int,None,None,...,NaN,None,None,None,None,None,None,None,None,None
1,MyDb,dbo,DaySpaVisit,CustomerVisitStart,2,None,NO,datetime2,None,None,...,7.0,None,None,None,None,None,None,None,None,None
2,MyDb,dbo,DaySpaVisit,CustomerVisitEnd,3,None,NO,datetime2,None,None,...,7.0,None,None,None,None,None,None,None,None,None
3,MyDb,dbo,DaySpaVisit,AmenityUseInMinutes,4,None,NO,int,None,None,...,NaN,None,None,None,None,None,None,None,None,None


## Maximum clients at one time

To determine how many staff they should have in the spa center, we will need to count the maximum number of client in the spa at any one time 

For example, the first customer are in the center from 1:55 PM until 3:55 PM. Then a group of 2 clients arrives at 4:00 PM before the first customer leaves, so the maximum at that point is 3.

To count the number of clients at one time, we will first break up our start and end times into separate rows so that we have an event per entrance and an event per exit. Plus we will add two columns: entry count and start ordinal. Entry count keep track of the number of people in the store at a given time and decrements whenever a person leaves. Start ordinal shows the order of entry, and NULL for exit.
First we will create a CTE named #StartStopPoints which is a self-union the DaySpaVisit table. Its left table consits  of entry data and its right table consist exit. 
Then we will create a temp table called #StartStopOrder from the CTE we just made. It takes each of the start and end times in the first query and adds a new ordinal value arranging when people leave and enter. We order by time of entry and then by start ordinal (start ordinal is important because exits marked as NULL values, so they will sort before the entrances. This way, if a person walks out the door exactly when another person walks in the door, we decrement the counter for the person leaving and then increment the counter for the person entering). 

In [61]:
#drop table in case the table already exitss
cursor = cnxn.cursor()
cursor.execute("Drop TABLE IF EXISTS #StartStopOrder")
cnxn.commit()
# Create #StartStopPoints and #StartStopOrder
cursor.execute("WITH #StartStopPoints AS ( \
                    SELECT dsv.CustomerVisitStart AS TimeUTC, \
                            1 AS EntryCount, \
                            Row_number() OVER ( ORDER BY dsv.CustomerVisitStart ) AS StartOrdinal \
                    FROM DaySpaVisit dsv \
                    UNION ALL \
                    SELECT dsv.CustomerVisitEnd AS TimeUTC, -1 AS EntryCount, NULL AS StartOrdinal \
                    FROM DaySpaVisit dsv)  \
                SELECT \
                    s.TimeUTC,   \
                    s.EntryCount,    \
                   s.StartOrdinal,    \
                   ROW_NUMBER() OVER (ORDER BY s.TimeUTC, s.StartOrdinal) AS StartOrEndOrdinal \
                INTO #StartStopOrder \
                FROM #StartStopPoints s ;")
cnxn.commit()
query = "SELECT * FROM #StartStopOrder"
x =pd.read_sql(query, cnxn)
x.head()

,TimeUTC,EntryCount,StartOrdinal,StartOrEndOrdinal
0,2018-12-18 13:55:00.0000000,1,1.0,1
1,2018-12-18 14:54:00.0000000,-1,NaN,2
2,2018-12-18 15:00:00.0000000,1,2.0,3
3,2018-12-18 15:00:00.0000000,1,3.0,4
4,2018-12-18 15:00:00.0000000,1,4.0,5


The StartEndOrdinal value gives us an ordering of the order in which people entered and left the spa. If we see positive EntryCount values start to outnumber negative counts, we have more people in the store.

To know the maximun number of people in the spa at any given time, we should count total of people at the time when people enter the spa (entrycount = 1). 
We have number of people inside = running total of entry - running total of exit in which : 
    - Running total of entry = StartOrdinal 
    - Running total of exit = Running total of entry and exist - runining total of entry = StartOrEndOrdinal - StartOrdinal
Therefore we can find the number of people at one given time by doubling the `StartOrdinal` value and then subtracting from it the `StartEndOrdinal`. Below is the 5 HIGHEST numbers

In [67]:
query = "SELECT TimeUTC , (2 * StartOrdinal - StartOrEndOrdinal) \
FROM #StartStopOrder \
WHERE EntryCount = 1\
ORDER BY (2 * StartOrdinal - StartOrEndOrdinal) DESC"

x =pd.read_sql(query, cnxn)
x.head()

,TimeUTC,
0,2018-12-18 15:00:00.0000000,90
1,2018-12-18 15:00:00.0000000,89
2,2018-12-18 15:00:00.0000000,88
3,2018-12-18 15:00:00.0000000,87
4,2018-12-18 15:00:00.0000000,86


To find the maximum of clients visit the spa at one time, we could Max command in SQL. 
However from the table above we can tell the maximun people in the spa at one time is 90

## Potential fraud

After showing the results to your team, a senior analyst raised a concern that the number is unsual high. She thinks that there might be a potential fraud in which a multiple people enter the spa using only one membership. 

We know that the company gives each customer one pass for personal use and a single guest pass. The  check-in and check-out data for each client and guest passes tie back to the base customer ID. Therefore we know that there might be overlap when a client and guest both check in together. However if there are more than 2 overlapping entries for a single client, it's a violation to our business rule.

We can identify the clients that violiated our rules by apply the similar method we applied previously. However we will look into each customer ID other than overrall customer base. 
To do so, we reuse the previous queries but in the Row_number commands we use 'PARTITION BY CustomerID' to count their entries seperately.

In [69]:
cursor = cnxn.cursor()
cursor.execute("Drop TABLE IF EXISTS #StartStopOrder")
cnxn.commit()

In [70]:
cursor = cnxn.cursor()
cursor.execute(" WITH #StartStopPoints AS ( \
                    SELECT dsv.CustomerID, dsv.CustomerVisitStart AS TimeUTC, 1 AS EntryCount, \
                        Row_number() OVER (PARTITION BY dsv.CustomerID ORDER BY dsv.CustomerVisitStart ) AS StartOrdinal \
                    FROM dbo.DaySpaVisit dsv \
                UNION ALL \
                    SELECT dsv.CustomerID, dsv.CustomerVisitEnd AS TimeUTC, -1 AS EntryCount, NULL AS StartOrdinal \
                    FROM dbo.DaySpaVisit dsv) \
                SELECT s.*, ROW_NUMBER() OVER ( PARTITION BY s.CustomerID ORDER BY s.TimeUTC, s.StartOrdinal) AS StartOrEndOrdinal \
                    INTO #StartStopOrder \
                FROM #StartStopPoints s ")
cnxn.commit()

In [36]:
query = "SELECT s.CustomerID, MAX(2 * s.StartOrdinal - s.StartOrEndOrdinal) AS MaxConcurrentCustomerVisits \
        FROM #StartStopOrder s \
        WHERE s.EntryCount = 1 \
        GROUP BY s.CustomerID \
        HAVING MAX(2 * s.StartOrdinal - s.StartOrEndOrdinal) > 2 \
        ORDER BY MaxConcurrentCustomerVisits desc"
x =pd.read_sql(query, cnxn)
x

,CustomerID,MaxConcurrentCustomerVisits
0,79,4
1,19,3
2,25,3
3,85,3
4,16,3
5,53,3
6,71,3
7,17,3
8,31,3
9,60,3


In [73]:
#total of customers
query = "SELECT COUNT(DISTINCT CustomerID) as total_customers FROM DaySpaVisit"

x =pd.read_sql(query, cnxn)
x.head()

,total_customers
0,90


We can conclude that there're 33 clients that violated our rule. Considering we only have 90 customers, this is a high number of violation!